# Imports

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import entropy, sem
from utils.supplementaryFunctions import *
from utils.plotSettings import *
parula = get_parula_cmap()
%matplotlib qt

# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import RBF, DotProduct

In [8]:
# gpdf = pd.read_csv(r"L:\4portProb_processed\gpdf_alphas.csv")
# gpdf.drop(columns = "Unnamed: 0", inplace = True)

# Parameter settings

In [3]:
# randomly sample numbers from given range 
# col = 'beta'
# list_params = np.random.uniform(0.1, 1, size = 5)
# np.random.seed(42)

# param_distributions = {
#     "alpha": loguniform(1e0, 1e3),
#     "kernel__length_scale": loguniform(1e-2, 1e2),
    # "kernel__periodicity": loguniform(1e0, 1e1),
# }
# # run the gp for all params 
# print(list(itertools.product(alphas, betas)))
# print(list_params)

# Sim runs (sklearn, gpr)

In [27]:
# set parameters
beta = 0.0067        # exploration coefficient
alpha = 34.725            # observation noise variance
ls = 1.25        # length scale of the RBF kernel; larger values = more spatial generalization
tau = 0.011       # when using softmax
scale = 0.0     # scale of Gumbel noise (set to 0 to turn off noise)
N = 4           # number of arms 
T = 100         # number of trials to simulate
q0 = 0.4617
gpdf = pd.DataFrame()
col = 'beta'
list_params = [0.0067]

# spatial location of arms (in 1D)
X = np.linspace(1,N,N).reshape(N,1)
for var in list_params:
    # which param to change? 
    beta = var
    # set up Gaussian process
    # this assumes a smooth kernel (only this rn)+ a linear kernel (not here)
    # you can turn either one off by removing that term
    # you can also weight the terms
    kernel = RBF(length_scale=ls)  #
    gp = GaussianProcessRegressor(kernel=kernel, alpha=alpha, optimizer=None)
    
    # Gaussian process prediction
    def gp_predict(r, x):
        gp.fit(x, r)
        m, sd = gp.predict(X, return_std=True)
        # print(x, r, m, sd)
        return m, sd
    
    # Gaussian process upper confidence bound policy
    def gp_ucb(m, sd):
        noise = np.random.gumbel(0, scale, m.size) # set to 0
        # a = np.argmax(m + beta*sd + noise)
        v = m + beta*sd + noise
        a = np.random.choice((np.where(v == np.amax(v))[0]))
        return a

    def gp_greedy(m, sd):
        noise = np.random.gumbel(0, scale, m.size)
        v = m
        a = np.random.choice((np.where(v == np.amax(v))[0]))
        return a

    def gp_softmax(m, sd):
        noise = np.random.gumbel(0, scale, m.size)
        v = m
        it = 0.1
        norm = np.sum(np.exp(v*it))
        p = [(np.exp(v[arm]*it))/norm for arm in range(N)]
        a = np.random.multinomial(1, p).nonzero()[0][0]
        return a

    def gp_ucb_softmax(m, sd):
        noise = np.random.gumbel(0, scale, m.size)
        p = np.exp((1/tau)*m + (beta/tau)*sd + noise)
        p = p/np.sum(p)
        a = np.random.multinomial(1, p).nonzero()[0][0]
        return a
    
    # simulator for a particular reward function
    # the simulator will choose actions according to the GP-UCB policy
    def gp_simulate(R):
        a = np.zeros(T, dtype=int)
        r = np.zeros(T)
        m = np.ones((T,N))*q0
        sd = np.zeros((T,N))+np.sqrt(alpha)
        
        for t in range(T):
            a[t] = gp_ucb_softmax(m[t,:], sd[t,:])    # choose action
            r[t] = R(a[t])                    # observe reward
            
            if t < T-1:
                m[t+1,:], sd[t+1,:] = gp_predict(r[:(t+1)], X[a[:(t+1)]]) # update posterior
                # print(X[a[:(t+1)]])
            # print(a[t], r[t], m[t], sd[t])   
        return a, r, m, sd
    
    # reward function
    # as an example, this creates linearly increasing Bernoulli reward probabilities across the action space
    # p = np.linspace(0,1,N)
    
    # my function, with permuted probabilities
    def fxn(mean, arms):
        x = np.linspace(1, arms, arms)
        sig = 1.75/2 #1.75/2 for 4 armed
        amp = 0.7
        vo = 0.1
        gx = (amp*np.exp(-0.5*((x-mean)**2)/(sig**2)))+vo
        gx = np.random.permutation(gx)
        return gx
    
    # run simulation and print results
    # notice that when using the linear (dot product) kernel, the policy immediately figures out to choose the last action
    
    cols = [col, 'session', 'port', 'reward', 'rewprob'] + [f'mu{i}' for i in range(1, N+1)] + [f'sd{i}' for i in range(1, N+1)]
    sessions = 1000
    window = 5
    ps = []
    # run gp simulator for some sessions...
    for sess in range(sessions):
    
        # new rew prob for bandit in each session
        p = fxn(np.random.choice(X.flatten()), N)
        ps.append(p)
        # print(p)
        # R fx
        def R(a):
            return np.random.binomial(1, p[a])
    
        # run
        a, r, m, sd = gp_simulate(R)
    
        # rwp for all a
        rwp = [p[a_row] for a_row in a]

        # take as many means as arms 
        #???
        
        # save data
        if N == 4:
            data = np.array([np.ones(T)*var, np.ones(T)*sess+1, a+1, r, rwp,
                             m.T[0], m.T[1], m.T[2], m.T[3],
                             sd.T[0], sd.T[1], sd.T[2], sd.T[3]]).T
        elif N == 8:
            data = np.array([np.ones(T)*var, np.ones(T)*sess+1, a, r, rwp,
                             m.T[0], m.T[1], m.T[2], m.T[3], m.T[4], m.T[5], m.T[6], m.T[7],
                             sd.T[0], sd.T[1], sd.T[2], sd.T[3], sd.T[4], sd.T[5], sd.T[6], sd.T[7]]).T
        tempdf = pd.DataFrame(data, columns = cols)
        gpdf = pd.concat([gpdf, tempdf], ignore_index = True)
    
    # format it to look like animal data 
    gpdf['choice_t1'] = gpdf.groupby('session').port.shift(-1, fill_value = 0)
    gpdf.loc[(gpdf.choice_t1 == 0), 'choice_t1'] = gpdf.loc[(gpdf.choice_t1 == 0), 'port']
    gpdf['choice_t2'] = gpdf.groupby('session').port.shift(-2, fill_value = 0)
    gpdf.loc[(gpdf.choice_t2 == 0), 'choice_t2'] = gpdf.loc[(gpdf.choice_t2 == 0), 'port']
    gpdf['shift_t0'] = (gpdf['choice_t1']==gpdf['port']).replace({True: 0, False: 1})
    gpdf['shift_t1'] = (gpdf['choice_t2']==gpdf['port']).replace({True: 0, False: 1})
    
    gpdf['regret'] = abs((gpdf.rewprob*100) - 80)
    gpdf['rr'] = (gpdf.groupby('session', as_index = False)
                .reward
                .rolling(window, center=True)
                .mean()
                .reward)
    gpdf['entropy'] = (gpdf.groupby('session', as_index = False)
                         .port
                         .rolling(window, center=True)
                         .apply(lambda x: entropy(calc_prob(x), base = 2))
                         .port)
    gpdf['trial'] = np.tile(np.arange(T), sessions)

    # potentially plot everything for this model, rr, entropy, tm, regret, distance, bias analysis, variability 
    fig = plt.figure(figsize = (8, 7))
    
    def avg_mat(df, col):
        g = df.groupby('session').cumcount()
        L = np.array(df.set_index(['session',g])
               .unstack(fill_value=0)
               .stack().groupby(level=0)
               .apply(lambda x: x[col].values.tolist())
               .tolist())
        return L
    
    secret_sauce = 'xkcd:cornflower'
    # figure 1.1 - regret across all sessions
    ax = plt.subplot(221)
    reg_mat = avg_mat(gpdf[gpdf[col] == var], 'regret')
    reg_mean = np.mean(reg_mat, axis = 0)
    reg_sem = sem(reg_mat, nan_policy = 'omit')
    ax.plot(reg_mean, color = secret_sauce)
    ax.fill_between(np.arange(reg_mat.shape[1]), reg_mean - reg_sem, reg_mean + reg_sem,  color = secret_sauce, alpha = 0.2)
    ax.set_title('Regret')
    
    # figure 1.2 - performance plot across all sessions
    ax = plt.subplot(222)
    rr_mat = avg_mat(gpdf[gpdf[col] == var], 'rr')
    rr_mean = np.mean(rr_mat, axis = 0)
    rr_sem = sem(rr_mat, nan_policy = 'omit')
    ax.plot(rr_mean, color = secret_sauce)
    ax.fill_between(np.arange(rr_mat.shape[1]), rr_mean - rr_sem, rr_mean + rr_sem,  color = secret_sauce, alpha = 0.2)
    ax.set_title('Reward rate')
    
    # figure 1.3 - entropy plot across all sessions
    ax = plt.subplot(223)
    entropy_mat = avg_mat(gpdf[gpdf[col] == var], 'entropy')
    entropy_mean = np.mean(entropy_mat, axis = 0)
    entropy_sem = sem(entropy_mat, nan_policy = 'omit')
    ax.plot(entropy_mean, color = secret_sauce)
    ax.fill_between(np.arange(entropy_mat.shape[1]), entropy_mean - entropy_sem,
                     entropy_mean + entropy_sem,  color = secret_sauce, alpha = 0.2)
    ax.set_title('Entropy')
    
    sns.despine()
    
    # figure 1.4 - transition matrix
    ax = plt.subplot(224)
    sns.heatmap(pd.crosstab(gpdf.port,
                            gpdf.choice_t1,
                            normalize = 'index', dropna = False),
                cmap = parula, annot = True, fmt = '.2f', vmin = 0.0,
                # vmax = 0.7,
                # mask = np.eye(N),
                square = True,
                xticklabels = np.arange(1,N+1), yticklabels = np.arange(1,N+1), ax = ax)
    # ax.patch.set_facecolor('white')
    # ax.set_title('Switch only transition matrix')
    
    plt.suptitle(f'{col} = {var}')
    plt.tight_layout()
    
    # generate figure 2 window
    fig = plt.figure(figsize = (8, 7))
    
    # figure 2.1 - all choice tm 
    ax = plt.subplot(221)
    sns.heatmap(pd.crosstab(gpdf[gpdf[col] == var].port, gpdf[gpdf[col] == var].choice_t1, normalize = 'index'),
                cmap = 'Greys', annot = True, fmt = '.2f', vmin = 0.0,
                # vmax = 0.7,
                square = True,
                xticklabels = np.arange(1,N+1), yticklabels = np.arange(1,N+1), ax = ax)
    ax.set_title('All choices')
    
    # figure 2.2 - rewarded tm 
    ax = plt.subplot(222)
    rewtm = pd.crosstab(gpdf[(gpdf.reward==1) & (gpdf[col] == var)].port,
                        gpdf[(gpdf.reward==1) & (gpdf[col] == var)].choice_t1, normalize = 'index')
    sns.heatmap(rewtm,
                cmap = 'Reds', annot = True, fmt = '.2f', vmin = 0.0,
                # vmax = 0.7,
                square = True,
                xticklabels = np.arange(1,N+1), yticklabels = np.arange(1,N+1), ax = ax)
    ax.set_title('Rewarded')
    
    # figure 2.3 - unrewarded tm
    ax = plt.subplot(223)
    unrewtm = pd.crosstab(gpdf[(gpdf.reward==0) & (gpdf[col] == var)].port,
                          gpdf[(gpdf.reward==0) & (gpdf[col] == var)].choice_t1, normalize = 'index')
    sns.heatmap(unrewtm,
                cmap = 'Blues', annot = True, fmt = '.2f', vmin = 0.0,
                # vmax = 0.7,
                square = True,
                xticklabels = np.arange(1,N+1), yticklabels = np.arange(1,N+1), ax = ax)
    ax.set_title('Unrewarded')
    
    # figure 2.4 - diff (rew, unrew) tm
    ax = plt.subplot(224)
    sns.heatmap(rewtm - unrewtm,
                cmap = 'coolwarm', annot = True, fmt = '.2f', center = 0,
                # vmin = 0.0,
                # vmax = 0.7,
                square = True,
                xticklabels = np.arange(1,N+1), yticklabels = np.arange(1,N+1), ax = ax)
    ax.set_title('Difference')
    
    plt.suptitle(f'{col} = {var}')
    plt.tight_layout()

    # generate figure 3 window
    fig = plt.figure(figsize = (8, 7))

    # figure 3.1 - movement when rew/unrew
    ax = plt.subplot(221)
    offset = 0.4
    # x = np.arange(4)
    gpdf['dist_abs'] = abs((gpdf.choice_t1+1) - (gpdf.port+1))
    y_pre_un = gpdf[(gpdf.reward == 0) & (gpdf[col] == var)].dist_abs.value_counts(normalize = True)
    y_pre_re = gpdf[(gpdf.reward == 1) & (gpdf[col] == var)].dist_abs.value_counts(normalize = True)
    ax.bar(y_pre_un.index+offset, y_pre_un, width = offset, color = 'b', label = 'Unrew')
    ax.bar(y_pre_re.index, y_pre_re, width = offset, color = 'r', label = 'Rew')
    ax.set_xlabel("Distance moved")
    ax.set_ylabel("Probablitity | Outcome")
    ax.set_yscale('log')
    ax.legend()
    sns.despine()

    plt.suptitle(f'{col} = {var}')
    plt.tight_layout()
    
# break

# gpdf.to_csv(r"L:\4portProb_processed\gpdf_ls.csv")


C:\Users\dlab\AppData\Local\Temp\ipykernel_4540\4261652056.py:139: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gpdf['shift_t0'] = (gpdf['choice_t1']==gpdf['port']).replace({True: 0, False: 1})
C:\Users\dlab\AppData\Local\Temp\ipykernel_4540\4261652056.py:140: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gpdf['shift_t1'] = (gpdf['choice_t2']==gpdf['port']).replace({True: 0, False: 1})
C:\Users\dlab\AppData\Local\Temp\ipykernel_4540\4261652056.py:162: FutureWarning: The previous implementation of stack is deprecated and will be r

In [6]:
import ipywidgets as widgets
from IPython.display import display
from matplotlib.patches import Rectangle
gpdf['trial'] = np.arange(gpdf.shape[0])
# Create a function to update the plot based on user input
def update_plot(session_range):
    sessdf_filtered = gpdf[(gpdf['session'].between(*session_range))]
    sessdf_filtered['port'] = sessdf_filtered['port']+1
    markers = {0: "$o$", 1: "o"}
#     markers = {0: 2, 1:3}
#     fig = plt.figure(figsize=(15, 8))
    g = sns.relplot(data=sessdf_filtered, y='port', x='trial', 
                    hue='rewprob',
                    aspect=11.7/5.5,
                    linewidth=0, s=100, palette = 'Blues', marker = '$o$',
                    markers=markers, style='reward')
                    # hue_norm=(-20, 80))
    axes = g.axes.flatten()
    last = gpdf.groupby('session')['trial'].max().to_list()
    start = gpdf.groupby('session')['trial'].min().to_list()
    # highport = [ind+1 for i in gpdf.groupby('session').head(1).rewprobfull for ind, j in enumerate(i) if j>=80]
    for ax in axes:
        for ind, l in enumerate(last):
            ax.axvline(l, linewidth=1, color='grey')
#             ax.axvline(, linewidth = 1, color = 'red')
            ax.set_yticks([1, 2, 3, 4])
            # ax.add_patch(Rectangle((start[ind]-1, highport[ind]-0.5), l-start[ind], 1, fc = 'xkcd:blue', alpha = 0.1))
    
    plt.ylim(0.5, 4.5)
    plt.title('Choices', y=1.05)
#     g._legend.set_title('Rew. %')
    g._legend.texts[0].set_text("Reward %")
    g._legend.texts[5].set_text("Outcome")
#     plt.xticks(ticks = [], labels = [])

    plt.xlabel(' ', labelpad=20)
    plt.ylabel('Port')
    
    # Calculate x-axis limits based on the displayed sessions
    x_limit = (sessdf_filtered['trial'].min(), sessdf_filtered['trial'].max())
    plt.xlim(*x_limit)
    sns.despine(bottom = True)
#     plt.savefig('C:/Users/dlab/OneDrive - Indian Institute of Science/Drawings/plots/15012024/eg.svg', dpi = 600)
    plt.show()

# Define the session range using a widget
session_range_slider = widgets.IntRangeSlider(
    value=[gpdf['session'].min(), gpdf['session'].max()],
    min=gpdf['session'].min(),
    max=gpdf['session'].max(),
    step=1,
    description='Session Range:',
    continuous_update=False
)

# Create an interactive plot using interactive
interactive_plot = widgets.interactive(update_plot, session_range=session_range_slider)

# Display the interactive plot
display(interactive_plot)
# plt.xticks(ticks = [], labels = [])
# plt.tight_layout()
# sns.set_context('talk')
# plt.title('Blissey - DLS then DMS lesion')

interactive(children=(IntRangeSlider(value=(1, 1000), continuous_update=False, description='Session Range:', m…

In [29]:
# potentially plot everything for this model, rr, entropy, tm, regret, distance, bias analysis, variability 
fig = plt.figure(figsize = (8, 7))

def avg_mat(df, col):
    g = df.groupby('session').cumcount()
    L = np.array(df.set_index(['session',g])
           .unstack(fill_value=0)
           .stack().groupby(level=0)
           .apply(lambda x: x[col].values.tolist())
           .tolist())
    return L

# figure 1.1 - regret across all sessions
ax = plt.subplot(221)
reg_mat = avg_mat(gpdf, 'regret')
reg_mean = np.mean(reg_mat, axis = 0)
reg_sem = sem(reg_mat, nan_policy = 'omit')
ax.plot(reg_mean, color = secret_sauce)
ax.fill_between(np.arange(reg_mat.shape[1]), reg_mean - reg_sem, reg_mean + reg_sem,  color = secret_sauce, alpha = 0.2)
ax.set_title('Regret')

# figure 1.2 - performance plot across all sessions
ax = plt.subplot(222)
rr_mat = avg_mat(gpdf, 'rr')
rr_mean = np.mean(rr_mat, axis = 0)
rr_sem = sem(rr_mat, nan_policy = 'omit')
ax.plot(rr_mean, color = secret_sauce)
ax.fill_between(np.arange(rr_mat.shape[1]), rr_mean - rr_sem, rr_mean + rr_sem,  color = secret_sauce, alpha = 0.2)
ax.set_title('Reward rate')

# figure 1.3 - entropy plot across all sessions
ax = plt.subplot(223)
entropy_mat = avg_mat(gpdf, 'entropy')
entropy_mean = np.mean(entropy_mat, axis = 0)
entropy_sem = sem(entropy_mat, nan_policy = 'omit')
ax.plot(entropy_mean, color = secret_sauce)
ax.fill_between(np.arange(entropy_mat.shape[1]), entropy_mean - entropy_sem,
                 entropy_mean + entropy_sem,  color = secret_sauce, alpha = 0.2)
ax.set_title('Entropy')

sns.despine()

# figure 1.4 - transition matrix
ax = plt.subplot(224)
sns.heatmap(pd.crosstab(gpdf.port, gpdf.choice_t1, normalize = 'index'),
            cmap = parula, annot = True, fmt = '.2f', vmin = 0.0,
            vmax = 0.3,
            # mask = np.eye(N),
            square = True,
            xticklabels = np.arange(1,N+1), yticklabels = np.arange(1,N+1), ax = ax)
# ax.patch.set_facecolor('white')
ax.set_title('Switch only transition matrix')

plt.suptitle(f'len_scale = {ls}')
plt.tight_layout()

# generate figure 2 window
fig = plt.figure(figsize = (8, 7))

# figure 2.1 - all choice tm 
ax = plt.subplot(221)
sns.heatmap(pd.crosstab(gpdf.port, gpdf.choice_t1, normalize = 'index'),
            cmap = 'Greys', annot = True, fmt = '.2f', vmin = 0.0,
            # vmax = 0.7,
            square = True,
            xticklabels = np.arange(1,N+1), yticklabels = np.arange(1,N+1), ax = ax)
ax.set_title('All choices')

# figure 2.2 - rewarded tm 
ax = plt.subplot(222)
rewtm = pd.crosstab(gpdf[gpdf.reward==1].port, gpdf[gpdf.reward==1].choice_t1, normalize = 'index')
sns.heatmap(rewtm,
            cmap = 'seismic', annot = True, fmt = '.2f', center = 0.0,
            # vmax = 0.7,
            square = True,
            xticklabels = np.arange(1,N+1), yticklabels = np.arange(1,N+1), ax = ax)
ax.set_title('Rewarded')

# figure 2.3 - unrewarded tm
ax = plt.subplot(223)
unrewtm = pd.crosstab(gpdf[gpdf.reward==0].port, gpdf[gpdf.reward==0].choice_t1, normalize = 'index')
sns.heatmap(unrewtm,
            cmap = 'seismic_r', annot = True, fmt = '.2f', center = 0.0,
            # vmax = 0.7,
            square = True,
            xticklabels = np.arange(1,N+1), yticklabels = np.arange(1,N+1), ax = ax)
ax.set_title('Unrewarded')

# figure 2.4 - diff (rew, unrew) tm
ax = plt.subplot(224)
sns.heatmap(rewtm - unrewtm,
            cmap = 'seismic', annot = True, fmt = '.2f', center = 0,
            # vmin = 0.0,
            # vmax = 0.7,
            square = True,
            xticklabels = np.arange(1,N+1), yticklabels = np.arange(1,N+1), ax = ax)
ax.set_title('Difference')

plt.suptitle(f'len scale = {ls}')
plt.tight_layout()
# break

C:\Users\dlab\AppData\Local\Temp\ipykernel_4540\3045736304.py:8: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack().groupby(level=0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_4540\3045736304.py:8: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack().groupby(level=0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_4540\3045736304.py:26: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  rr_sem = sem(rr_mat,

In [14]:
# distance traveled in the first n trials
fig = plt.figure(figsize = (10, 5))

def calc_dist_metric(tempdf, mask):
    mean_distance = pd.Series([2, 4/3, 4/3, 2], index=[1, 2, 3, 4])

    filtered = tempdf[mask]

    # calculate `d_value` for each group
    d_values = filtered.groupby('session')['disp'].apply(lambda x: x.abs().mean())

    # calculate `d_chance` for each group
    d_chances = filtered.groupby('session').apply(
        lambda group: np.sum(group['port'].value_counts(normalize=True) * mean_distance)
    )

    tempdf['d_value'] = tempdf.set_index('session').index.map(d_values).values
    tempdf['d_chance'] = tempdf.set_index('session').index.map(d_chances).values
    return tempdf


for enum, trial_group in enumerate(range(0, 101, 10)):    
    if trial_group == 0:
        continue
    ax = plt.subplot(2, 5, enum)

    tempdf = gpdf[(gpdf.trial.isin(np.arange(trial_group-9, trial_group)))]
    # compute switch probability after outcome
    rdf = calc_dist_metric(tempdf, mask = (tempdf.reward == 1) & (tempdf.shift_t0 == 1))
    y = rdf.d_value.mean()
    yerr = rdf.d_value.sem()
    chan = rdf.d_chance.mean()
    ax.bar(0, chan, color = 'grey', alpha = 0.2)
    ax.bar(0, y, color = 'red', yerr = yerr, label = 'R')

    nrdf = calc_dist_metric(tempdf, mask = (tempdf.reward == 0) & (tempdf.shift_t0 == 1))
    y = nrdf.d_value.mean()
    yerr = nrdf.d_value.sem()
    chan = nrdf.d_chance.mean()
    ax.bar(1, chan, color = 'grey', alpha = 0.2)
    ax.bar(1, y, color = 'blue', yerr = yerr, label = 'NR')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticks([0, 1], ['R', 'NR'], fontsize = 'large', color = 'grey')
    ax.set_ylim(1, 1.7)
    ax.set_title(f'Trials {trial_group-9} to {trial_group}')
# fig.supxlabel('Outcome at trial t')
plt.legend()
fig.supylabel('Average switch distance at trial t+1')
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_4540\3500392624.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby('session').apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_4540\3500392624.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempdf['d_value'] = tempdf.set_index('session').index.map(d_values).values
C:\Users\dlab\AppData\Local\Temp\ipykernel_4540\3500392624.py:18: SettingWithCopyWarning: 
A value is trying to be 

ValueError: Data must be 1-dimensional, got ndarray of shape (0, 23) instead

In [31]:
sns.set_style('whitegrid')
sns.set_context('notebook')
plt.figure(figsize = (4.5, 5))
gpdf['disp'] = gpdf['choice_t1']-gpdf['port']
gpdf['trial'] = np.tile(np.arange(T), sessions)
ax = plt.subplot(111)
rdf = calc_dist_metric(gpdf, mask = (gpdf.reward == 1) & (gpdf.shift_t0 == 1))
nrdf = calc_dist_metric(gpdf, mask = (gpdf.reward == 0) & (gpdf.shift_t0 == 1))
y = rdf.d_value.mean()
yerr = rdf.d_value.sem()
chan = rdf.d_chance.mean()
ax.bar(0, chan, color = 'grey', alpha = 0.2)
ax.bar(0, y, color = 'red', yerr = yerr, label = 'R')
y = nrdf.d_value.mean()
yerr = nrdf.d_value.sem()
chan = nrdf.d_chance.mean()
ax.bar(1, chan, color = 'grey', alpha = 0.2)
ax.bar(1, y, color = 'blue', yerr = yerr, label = 'NR')
plt.xticks([0, 1], ['R', 'NR'], fontsize = 'large', color = 'grey')
plt.yticks(color = 'grey', fontsize = 'large')
plt.ylabel('Average switch distance', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_4540\3500392624.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby('session').apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_4540\3500392624.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby('session').apply(


In [16]:
fig = plt.figure(figsize = (10, 5))

for enum, trial_group in enumerate(range(0, 101, 10)):
    
    if trial_group == 0:
        continue
    ax = plt.subplot(2, 5, enum)
    
    temperdf = gpdf[(gpdf.trial.isin(np.arange(trial_group-9, trial_group)))]
    # compute switch probability after outcome
    ax.bar(0, temperdf[(temperdf.reward == 1)].shift_t0.mean(),
           color = 'red', 
           yerr = temperdf[(temperdf.reward == 1)].shift_t0.sem(), 
           label = 'R')
    ax.bar(1, temperdf[(temperdf.reward == 0)].shift_t0.mean(), 
           color = 'blue', 
           yerr = temperdf[(temperdf.reward == 0)].shift_t0.sem(),
           label = 'NR')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticks([0, 1], ['R', 'NR'], fontsize = 'large', color = 'grey')
    ax.set_ylim(0, 0.4)
    ax.set_title(f'Trials {trial_group-9} to {trial_group}')

# fig.supxlabel('Outcome at trial t')
plt.legend()
fig.supylabel('Switch probability at trial t+1')
sns.despine()
plt.tight_layout()

In [32]:
plt.figure(figsize = (4.5, 5))
# yerr = tempdf[(tempdf.reward == 1) & (tempdf.shift_t0 == 1)].d_value.mean()
plt.bar([0], gpdf[(gpdf.reward == 1)].shift_t0.mean(), color = 'red')
plt.bar([1], gpdf[(gpdf.reward == 0)].shift_t0.mean(), color = 'blue')
plt.xticks([0, 1], ['R', 'NR'], fontsize = 'large', color = 'grey')
plt.yticks(color = 'grey', fontsize = 'large')
plt.ylabel('Average switch probability', fontsize = 'x-large')
plt.tight_layout()

# &Delta;Q analysis

In [145]:
first10 = gpdf[gpdf.ls == 2].groupby('session').head(10)
qvec_cols = ['mu1', 'mu2', 'mu3', 'mu4', 'mu5', 'mu6', 'mu7', 'mu8']
ind = first10['port'].astype(int)
delq_vec = first10.groupby('session')[qvec_cols].diff().to_numpy()
refq_vec = [delq_vec[row, i] for row, i in enumerate(ind)]

# how does alg update q values depending on distance
# how far from ref arm are you? take del q of that
delq_dist1, delq_dist2, delq_dist3 = [], [], []
delq_dist4, delq_dist5, delq_dist6 = [], [], []
delq_dist7 = []
for row, refarm in enumerate(ind):
    if refarm == 0:
        delq_dist1.append(delq_vec[row, 1])
        delq_dist2.append(delq_vec[row, 2])
        delq_dist3.append(delq_vec[row, 3])
        delq_dist4.append(delq_vec[row, 4])
        delq_dist5.append(delq_vec[row, 5])
        delq_dist6.append(delq_vec[row, 6])
        delq_dist7.append(delq_vec[row, 7])
    if refarm == 1:
        delq_dist1.append(delq_vec[row, 0])
        delq_dist1.append(delq_vec[row, 2])
        delq_dist2.append(delq_vec[row, 3])
        delq_dist3.append(delq_vec[row, 4])
        delq_dist4.append(delq_vec[row, 5])
        delq_dist5.append(delq_vec[row, 6])
        delq_dist6.append(delq_vec[row, 7])
    if refarm == 2:
        delq_dist1.append(delq_vec[row, 1])
        delq_dist1.append(delq_vec[row, 3])
        delq_dist2.append(delq_vec[row, 0])
        delq_dist2.append(delq_vec[row, 4])
        delq_dist3.append(delq_vec[row, 5])
        delq_dist4.append(delq_vec[row, 6])
        delq_dist5.append(delq_vec[row, 7])
    if refarm == 3:
        delq_dist1.append(delq_vec[row, 2])
        delq_dist2.append(delq_vec[row, 1])
        delq_dist3.append(delq_vec[row, 0])

# put it in an array acc to distance moved, based on rew/nr
ax = plt.subplot(221)
sns.boxplot([refq_vec, delq_dist1, delq_dist2, delq_dist3], whis = (0,100))

qvec_cols = ['mu1', 'mu2', 'mu3', 'mu4']
ind = first10[first10.reward==1]['port'].astype(int)
delq_vec = first10.groupby('session')[qvec_cols].diff()[first10.reward==1].to_numpy()
refq_vec = [delq_vec[row, i] for row, i in enumerate(ind)]

delq_dist1, delq_dist2, delq_dist3 = [], [], []
for row, refarm in enumerate(ind):
    if refarm == 0:
        delq_dist1.append(delq_vec[row, 1])
        delq_dist2.append(delq_vec[row, 2])
        delq_dist3.append(delq_vec[row, 3])
    if refarm == 1:
        delq_dist1.append(delq_vec[row, 0])
        delq_dist1.append(delq_vec[row, 2])
        delq_dist2.append(delq_vec[row, 3])
    if refarm == 2:
        delq_dist1.append(delq_vec[row, 1])
        delq_dist1.append(delq_vec[row, 3])
        delq_dist2.append(delq_vec[row, 0])
    if refarm == 3:
        delq_dist1.append(delq_vec[row, 2])
        delq_dist2.append(delq_vec[row, 1])
        delq_dist3.append(delq_vec[row, 0])

# put it in an array acc to distance moved, based on rew/nr
ax = plt.subplot(222)
sns.boxplot([refq_vec, delq_dist1, delq_dist2, delq_dist3], whis = (0,100))

qvec_cols = ['mu1', 'mu2', 'mu3', 'mu4']
ind = first10[first10.reward==0]['port'].astype(int)
delq_vec = first10.groupby('session')[qvec_cols].diff()[first10.reward==0].to_numpy()
refq_vec = [delq_vec[row, i] for row, i in enumerate(ind)]

delq_dist1, delq_dist2, delq_dist3 = [], [], []
for row, refarm in enumerate(ind):
    if refarm == 0:
        delq_dist1.append(delq_vec[row, 1])
        delq_dist2.append(delq_vec[row, 2])
        delq_dist3.append(delq_vec[row, 3])
    if refarm == 1:
        delq_dist1.append(delq_vec[row, 0])
        delq_dist1.append(delq_vec[row, 2])
        delq_dist2.append(delq_vec[row, 3])
    if refarm == 2:
        delq_dist1.append(delq_vec[row, 1])
        delq_dist1.append(delq_vec[row, 3])
        delq_dist2.append(delq_vec[row, 0])
    if refarm == 3:
        delq_dist1.append(delq_vec[row, 2])
        delq_dist2.append(delq_vec[row, 1])
        delq_dist3.append(delq_vec[row, 0])

# put it in an array acc to distance moved, based on rew/nr
ax = plt.subplot(223)
sns.boxplot([refq_vec, delq_dist1, delq_dist2, delq_dist3], whis = (0,100))

IndexError: index 7 is out of bounds for axis 1 with size 4

In [288]:
plt.plot(first10[first10.session == 1][qvec_cols])
plt.legend(qvec_cols)

In [103]:
first10[first10.session == 1].groupby('port').rewprob.value_counts()

port  rewprob 
0.0   0.800000    3
2.0   0.464315    1
3.0   0.151359    1
Name: count, dtype: int64

# Linear regression

In [188]:
from regression20241117 import *
gpdf['animal'] = 'gp_ucb'
gpdf['task'] = 'unstr'

hist = 6
data_unstr = data_prep(gpdf[gpdf.ls == 2], hist = hist, trialsinsess = 100, task = 'unstr', head = True)
for i in range(1,hist):
    data_unstr[f'ct{i}_rt{i}'] = data_unstr[f'ct{i}']*data_unstr[f'rt{i}']

In [189]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, root_mean_squared_error
from sklearn.linear_model import LinearRegression

from pandas.api.types import CategoricalDtype
# catdtype = CategoricalDtype(categories=np.arange(1,5), ordered = False) predicting numeric values

crs = [f'ct{i}_rt{i}' for i in range(1,hist)]
rs = [f'rt{i}' for i in range(1,hist)]
cs = [f'ct{i}' for i in range(1,hist)]

# define model vars
x_cols = cs+rs+crs
y_cols = ['ct0']

# which models/sims?
an_list = ['gp_ucb']

# fit model separately for each animal/simulation
ind = 1
# fig = plt.figure(figsize = (20, 10))
rmse_all = []
for an in an_list:
    y = data_unstr.groupby('animal').get_group(an)[y_cols].astype(float).to_numpy().flatten()
    X = data_unstr.groupby('animal').get_group(an)[x_cols].reset_index().drop(columns = 'index')

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
            
    lr = LinearRegression(fit_intercept = True)
    lr.fit(X_train, y_train)
    
    cols = lr.feature_names_in_
    # ind = lr.classes_
    #         ind = [1.0]
            
    score = lr.score(X_test, y_test)
    
    # y_pred_proba = lr.predict_proba(X_train)
    y_pred = lr.predict(X_train)
    
    # ll_null = log_loss(y_train, [calc_prob(y_train)]*len(y_train), labels = [1,2,3,4])
    # ll_model = log_loss(y_train, y_pred_proba, labels = [1,2,3,4])
    
    # ll_null = log_loss(y_train, [calc_prob(y_train)]*len(y_train))
    # ll_model = log_loss(y_train, y_pred_proba)
    # pseudo_r2 = (ll_null - ll_model) / ll_null
    rmse = root_mean_squared_error(y_train, y_pred)
    rmse_all.append(rmse)
    print(f'{an}, RMSE = {round(rmse, 5)}, Score = {round(score, 5)}')
    
    ax = plt.subplot(111)
    ax.bar(['intercept'] + x_cols, np.hstack([lr.intercept_, lr.coef_]))
    ax.set_title(f'{an}, RMSE={round(rmse, 2)}')
    ind+=1
sns.despine()
plt.tight_layout()

gp_ucb, RMSE = 0.37022, Score = 0.90313


C:\Users\dlab\AppData\Local\Temp\ipykernel_3492\1742289879.py:51: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  ax = plt.subplot(111)
C:\Users\dlab\AppData\Local\Temp\ipykernel_3492\1742289879.py:56: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


# Something new

In [6]:
def logFunc(x, a, b):
    return a + b*np.log(x)
T = 100
unstr_var = np.array([0.36322384, 0.05456885])

In [91]:
from sklearn.metrics import root_mean_squared_error
ax = plt.subplot(111)
rr_mat = avg_mat(gpdf[gpdf.ls == 0.1], 'rr')
rr_mean = np.mean(rr_mat, axis = 0)
rr_sem = sem(rr_mat, nan_policy = 'omit')
ax.plot(rr_mean, color = 'xkcd:azure')
ax.plot(logFunc(np.arange(1, T+1), *unstr_var), linestyle = 'dotted')
ax.fill_between(np.arange(rr_mat.shape[1]), rr_mean - rr_sem, rr_mean + rr_sem,  color = 'xkcd:azure', alpha = 0.2)
ax.set_title('Reward rate')
rmse_beta = []
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
for i in gpdf.ls.unique():
    rr_mat = avg_mat(gpdf[gpdf.ls == i], 'rr')
    rr_mean = np.nanmean(rr_mat, axis = 0)
    try:
        rmse_temp = rmse(logFunc(np.arange(1, T+1), *unstr_var), rr_mean)
        rmse_beta.append([i, rmse_temp])
    except:
        continue


C:\Users\dlab\AppData\Local\Temp\ipykernel_3492\1489734257.py:15: RuntimeWarning: Mean of empty slice
  rr_mean = np.nanmean(rr_mat, axis = 0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_3492\1489734257.py:15: RuntimeWarning: Mean of empty slice
  rr_mean = np.nanmean(rr_mat, axis = 0)


In [96]:
ax = plt.subplot(111)
A = np.array(rmse_beta)[:, 0]
B = np.array(rmse_beta)[:, 1]
ax.plot(A, B, 'o')
for x, y in zip(A, B):                                       # <--
    ax.text(x, y, f"({round(x, 2)}, {round(y, 2)})", fontsize=10) # <--

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


In [71]:
rr_mat = avg_mat(gpdf[gpdf.beta == 1], 'rr')
rr_mean = np.nanmean(rr_mat, axis = 0)
plt.plot(logFunc(np.arange(1, T+1), *unstr_var))
plt.plot(rr_mean)


# sklearn GPC

In [ ]:
def exponential_cov(x, y, scale, length_scale):
    return scale * np.exp(
        -0.5 * length_scale * np.subtract.outer(x, y)**2
    )

def conditional(x_new, x, y, scale, length_scale):
    B = exponential_cov(x_new, x, scale, length_scale)
    C = exponential_cov(x, x, scale, length_scale)
    A = exponential_cov(x_new, x_new, scale, length_scale)
    mu = np.linalg.inv(C).dot(B.T).T.dot(y)
    sigma = A - B.dot(np.linalg.inv(C).dot(B.T))
    return (mu.squeeze(), sigma.squeeze())

def predict(x, data, kernel, scale, length_scale, sigma, t):
    k = [kernel(x, y, scale, length_scale) for y in data]
    Sinv = np.linalg.inv(sigma)
    y_pred = np.dot(k, Sinv).dot(t)
    sigma_new = kernel(x, x, scale, length_scale) - np.dot(k, Sinv).dot(k)
    return y_pred, sigma_new


In [2]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, DotProduct
import matplotlib.pyplot as plt
from utils.plotSettings import *

In [3]:
# set parameters array([0.23323558, 0.14652085, 0.76057374])
beta = 0        # ucb
alpha = 0.      # observation noise variance
ls = 1.0        # length scale of the RBF kernel; larger values = more spatial generalization
scale = 0.0     # scale of Gumbel noise (set to 0 to turn off noise)
N = 4           # number of arms
T = 10          # number of trials to simulate

beta = 0.23323558
temp = 0.14652085
ls = 0.76057374

In [4]:
def fxn(mean, arms, permute = False):
    x = np.linspace(1, arms, arms)
    sig = 1.75/2
#     amp = 1/(sig*np.sqrt(2*np.pi))
    amp = 0.7
    vo = 0.1
    gx = (amp*np.exp(-0.5*((x-mean)**2)/(sig**2)))+vo
    if permute:
        gx = np.random.permutation(gx)
    return gx
################ GIVING REWARD #####################
def rewarding(prob, reward_val):
    temp = reward_val
    rand = np.random.uniform(0, 1)
    return temp if rand <= prob else 0

In [22]:
import numpy as np
from scipy.special import expit
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Product, ConstantKernel

def softmax(arr, temp):
    P = np.exp(arr*1/temp)
    P = P/sum(P)
    return P

# x0 = np.array([0.23323558, 0.14652085, 0.76057374])
# x0 = [0.13455899, 0.13112178, 0.66679744]
# x0 = [0.22396454, 0.0706229, 0.92816823]
x0 = [0, 1/2.6748, 0.9727]

# x0 = np.array([0.00000000e+00, 1.50000000e-03, 1.85671396e+00])
trials = 100
sessions = 1000
arms = 4
ls = x0[2]
# ls = 1
# n_samples = 500
be = x0[0]
# be = 2
temp = x0[1]
np.random.seed(42)
a = np.zeros((sessions, trials))
r = np.zeros((sessions, trials))
p_est = np.zeros((sessions, trials, arms), dtype = float)
prob_mean = np.ones((sessions, trials, arms), dtype = float)*np.nan
prob_std = np.zeros((sessions, trials, arms), dtype = float)
rp_set = [fxn(np.random.randint(1, arms+1), arms, True) for i in range(10000)]
X = np.arange(arms).reshape(-1, 1) # excuse sklearn for using this dumb syntax
knl = Product(RBF(length_scale=ls), ConstantKernel(constant_value=1**2))
for sess in range(sessions):
    # reward probability changes here
    rp = rp_set[sess]
    # reset model?
    gp = GaussianProcessClassifier(kernel=knl, optimizer = None)

    for t in range(trials):
        if len(np.unique(r[sess, :t])) > 1:
            # update gp using all the info we got so far on actions and rew
            gp.fit(X[a[sess, :t].astype(int)], r[sess, :t])

            # draw arm using UCB
            # get latent mean and variance
            mu, var = gp.latent_mean_and_variance(X)
            prob_mean[sess, t, :] = expit(mu)
            prob_std[sess, t, :] = expit(np.sqrt(var))
        
            # compute UCB
            input_to_softmax = prob_mean[sess, t, :] + be*prob_std[sess, t, :]
            p_est[sess, t, :] = softmax(input_to_softmax - max(input_to_softmax), temp = temp)
            # p_est[sess, t, :] = softmax(input_to_softmax, temp = temp)
            # select arm using this prob
            a[sess, t] = np.random.multinomial(1, p_est[sess, t, :]).nonzero()[0][0]
            chosen = int(a[sess, t])

            # reward chosen arm
            r[sess, t] = rewarding(rp[chosen], 1)

        else:
            a[sess, t] = np.random.choice(np.arange(arms))
            chosen = int(a[sess, t])
            r[sess, t] = rewarding(rp[chosen], 1)

In [ ]:
%matplotlib qt
# from utils.plotSettings import *
# for every chosen action, get q value
curr_q = prob_mean[np.arange(sessions)[:, None], np.arange(trials)[None, :], a.astype(int)]
# get the previous q_value for each of these arms
prev_q = np.full_like(curr_q, np.nan)
prev_q[:, 1:] = curr_q[:, :-1]  
# rpe is diff between value of chosen arm - value on prev trial
rpe = curr_q - prev_q
# calc rpe for arm when it was rewarded t-1, and nr t
prev_r = np.full_like(r, np.nan)
prev_r[:, 1:] = r[:, :-1]
curr_r = r

dist_moved = np.full_like(a, np.nan)
dist_moved[:, 1:] = np.abs(a[:, 1:] - a[:, :-1])

prev_a = np.full_like(a, np.nan)
prev_a[:, 1:] = a[:, :-1]

rpe_sub = rpe[:, 1:]
# subset rpe for every condition
rn = rpe[(prev_r == 1) & (curr_r == 0)]
rr = rpe[(prev_r == 1) & (curr_r == 1)]
nn = rpe[(prev_r == 0) & (curr_r == 0)]
nr = rpe[(prev_r == 0) & (curr_r == 1)]
# plot the mean across all sessions and trials
plt.figure()
plt.bar(np.arange(4), np.array([np.nanmean(nr), np.nanmean(rr), np.nanmean(nn), np.nanmean(rn)]))
plt.xticks(np.arange(4), ['NR', 'RR', 'NN', 'RN'])

([<matplotlib.axis.XTick at 0x1b9d228fc70>,
 [Text(0, 0, 'NR'), Text(1, 0, 'RR'), Text(2, 0, 'NN'), Text(3, 0, 'RN')])

In [23]:
# to identify if model generalizes or not:
next_mean = np.full_like(prob_mean, np.nan)
next_mean[:, :-1, :] = prob_mean[:, 1:, :]
rpe_mat = next_mean - prob_mean

l = []
n_sessions, n_trials, n_actions = rpe_mat.shape

for distance in [0, 1, 2]:
    # For each trial, get the RPE at chosen arm ± distance
    rpe_at_dist = np.full((n_sessions, n_trials), np.nan)
    for sess in range(n_sessions):
        for t in range(n_trials):
            idx = int(a[sess, t])
            # Only use valid indices
            target_idx = idx + distance
            if 0 <= target_idx < n_actions:
                rpe_at_dist[sess, t] = rpe_mat[sess, t, target_idx]
    # Now analyze by reward outcome
    l.append(np.nanmean(rpe_at_dist[r == 1]))
    l.append(np.nanmean(rpe_at_dist[r == 0]))
plt.figure()
plt.plot(l[::2], label='R', color='xkcd:red', marker='o', lw = 3)
plt.plot(l[1::2], label='N', color='xkcd:blue', marker='o', lw = 3)
plt.axhline(0, color = 'k', lw = 0.75, ls = '-')
plt.xticks(np.arange(3), np.arange(3))
plt.ylabel('RPE')
plt.xlabel('Distance from chosen action')
plt.legend()
plt.tight_layout()

In [24]:
# RPE at different distances from chosen port
n_sessions = prob_mean.shape[0]
n_trials = prob_mean.shape[1]

curr_a = a
prev_a = np.full_like(curr_a, np.nan)
prev_a[:, 1:] = curr_a[:, :-1]
dist_moved = np.abs(curr_a - prev_a)

valid_mask = np.ones_like(curr_a, dtype=bool)
valid_mask[:, 0] = False

next_mean = np.full_like(prob_mean, np.nan)
next_mean[:, :-1, :] = prob_mean[:, 1:, :]
curr_q = prob_mean[np.arange(n_sessions)[:, None], np.arange(n_trials)[None, :], curr_a.astype(int)]
next_q = next_mean[np.arange(n_sessions)[:, None], np.arange(n_trials)[None, :], curr_a.astype(int)]

curr_r = r
prev_r = np.full_like(curr_r, np.nan)
prev_r[:, 1:] = curr_r[:, :-1]

l = []
len_sess = 100
# now compute difference between each of these trials, and condition by distance and by outcome condition
for distance in [0, 1, 2]:
    mask_NR = (curr_r == 1) & (prev_r == 0) & (dist_moved == distance) #& valid_mask
    mask_RR = (curr_r == 1) & (prev_r == 1) & (dist_moved == distance) #& valid_mask
    mask_RN = (curr_r == 0) & (prev_r == 1) & (dist_moved == distance) #& valid_mask
    mask_NN = (curr_r == 0) & (prev_r == 0) & (dist_moved == distance) #& valid_mask
    # l.append(np.nanmean((curr_r - curr_q)[:, :len_sess][mask_NR[:, :len_sess]]))
    # l.append(np.nanmean((curr_r - curr_q)[:, :len_sess][mask_RR[:, :len_sess]]))
    # l.append(np.nanmean((curr_r - curr_q)[:, :len_sess][mask_RN[:, :len_sess]]))
    # l.append(np.nanmean((curr_r - curr_q)[:, :len_sess][mask_NN[:, :len_sess]]))

    l.append(np.nanmean(curr_r[mask_NR[:, :len_sess]] - curr_q[mask_NR[:, :len_sess]]))
    l.append(np.nanmean(curr_r[mask_RR[:, :len_sess]] - curr_q[mask_RR[:, :len_sess]]))
    l.append(np.nanmean(curr_r[mask_RN[:, :len_sess]] - curr_q[mask_RN[:, :len_sess]]))
    l.append(np.nanmean(curr_r[mask_NN[:, :len_sess]] - curr_q[mask_NN[:, :len_sess]]))
    print("NR:", np.sum(mask_NR), "RR:", np.sum(mask_RR), "RN:", np.sum(mask_RN), "NN:", np.sum(mask_NN))

# from utils.plotSettings import *
plt.figure()
plt.plot(l[::4], 'o-', label= 'NR', color = 'xkcd:maroon')
plt.plot(l[1::4], 'o-', label = 'RR', color = 'xkcd:red')
plt.plot(l[2::4], 'o-', label = 'RN', color = 'xkcd:light blue')
plt.plot(l[3::4], 'o-', label = 'NN', color = 'xkcd:blue')
plt.xlabel(r'$|a_{t} - a_{t-1}|$')
plt.axhline(0, color = 'k', ls = '--')
plt.xticks(np.arange(3), np.arange(3))
plt.ylabel(r'$\mu_{t+1, a_{t}} - \mu_{t, a_{t}}$')
plt.legend(title = r'$r_{t-1}r_{t}$')
# plt.ylim(-1, 0.5)
plt.tight_layout()
plt.show()

NR: 5092 RR: 12728 RN: 5558 NN: 6057
NR: 10215 RR: 7606 RN: 10038 NN: 8309
NR: 6112 RR: 3835 RN: 5822 NN: 5956


C:\Users\dlab\AppData\Local\Temp\ipykernel_17852\1942099084.py:54: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


np.float64(-0.4856951425511944)

In [20]:
np.nanmean(l[3::4])

np.float64(-0.561035362721252)

In [37]:
plt.figure(figsize = (9, 7.5))
colors = {0: "#fce044f8", 1: '#2eafd6', 2:'#4c57f4'}
# plot transition matrix
ax = plt.subplot(221)
parula = get_parula_cmap()

sns.heatmap(pd.crosstab(pd.Series(a.flatten()).shift(-1), a.flatten(), normalize= 'index'),
            cmap = parula,
            square=True, 
            # annot=True,
            vmax = 0.25,
            xticklabels=np.arange(1,5), 
            yticklabels=np.arange(1,5), ax=ax)
ax.set_xlabel('Choice at t+1')
ax.set_ylabel('Choice at t')

# get rpe for switches
width = 0.2
gap = 0.2
for dist in [0, 1, 2]:
    ax = plt.subplot(222)
    # subset rpe for every condition
    sw_dist = np.abs(np.diff(a, append=np.nan))[:, 1:]
    rn = rpe_sub[(prev_r == 1) & (curr_r == 0) & (sw_dist==dist)]
    rr = rpe_sub[(prev_r == 1) & (curr_r == 1) & (sw_dist==dist)]
    nn = rpe_sub[(prev_r == 0) & (curr_r == 0) & (sw_dist==dist)]
    nr = rpe_sub[(prev_r == 0) & (curr_r == 1) & (sw_dist==dist)]
    # plot the mean across all sessions and trials
    ax.bar(np.arange(4)+width*dist, np.array([np.nanmean(nr), np.nanmean(rr), np.nanmean(nn), np.nanmean(rn)]), 
           label = dist, 
           width=width,
           color = colors[dist])
    ax.set_xticks(np.arange(4), ['NR', 'RR', 'NN', 'RN'])
ax.set_ylabel('Mean RPE')
leg = ax.legend(frameon=False, title = 'Distance')
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

# get rpe difference between conditions
for dist in [0, 1, 2]:
    ax = plt.subplot(223)
    # subset rpe for every condition
    sw_dist = np.abs(np.diff(a, append=np.nan))[:, 1:]
    rn = rpe_sub[(prev_r == 1) & (curr_r == 0) & (sw_dist==dist)]
    rr = rpe_sub[(prev_r == 1) & (curr_r == 1) & (sw_dist==dist)]
    nn = rpe_sub[(prev_r == 0) & (curr_r == 0) & (sw_dist==dist)]
    nr = rpe_sub[(prev_r == 0) & (curr_r == 1) & (sw_dist==dist)]
    # plot the mean across all sessions and trials
    ax.bar(np.arange(2)+width*dist, np.array([np.nanmean(nr) - np.nanmean(rr), np.nanmean(rn) -np.nanmean(nn)]),
           label = dist, 
           width=width, 
           color = colors[dist])
    ax.set_xticks(np.arange(2)+width, ['NR - RR', 'RN - NN'])
ax.set_ylabel('Change in RPE')
leg = ax.legend(frameon=False, title = 'Distance')
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

#get rpe difference#
for dist in [0, 1, 2]:
    ax = plt.subplot(224)
    # subset rpe for every condition
    sw_dist = np.abs(np.diff(a, append=np.nan))[:, 1:]
    rn = rpe_sub[(prev_r == 1) & (curr_r == 0) & (sw_dist==dist)]
    rr = rpe_sub[(prev_r == 1) & (curr_r == 1) & (sw_dist==dist)]
    nn = rpe_sub[(prev_r == 0) & (curr_r == 0) & (sw_dist==dist)]
    nr = rpe_sub[(prev_r == 0) & (curr_r == 1) & (sw_dist==dist)]
    # plot the mean across all sessions and trials
    ax.bar(np.arange(1)+width*dist+gap*dist, np.array((np.nanmean(nr) - np.nanmean(rr))-(np.nanmean(rn) -np.nanmean(nn))), 
           label = dist, 
           width=width, 
           color = colors[dist])
ax.set_xticks(np.arange(3)*(width+gap), [0,1,2])
ax.set_ylabel('Change in RPE')
ax.set_xlabel('Distance from chosen arm')
leg = ax.legend(frameon=False, title = 'Distance')
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())
plt.tight_layout()

ValueError: operands could not be broadcast together with shapes (1000,100) (1000,99) 

In [254]:
# reward rate
%matplotlib qt
from utils.plotSettings import *
from scipy.stats import sem
plt.figure()
plt.plot(np.mean(r, axis = 0), color = 'xkcd:cornflower')
plt.fill_between(np.arange(trials), np.mean(r, axis = 0) - sem(r, axis = 0), np.mean(r, axis = 0) + sem(r, axis = 0), alpha = 0.3, color = 'xkcd:cornflower')
chance_level = np.mean(rp_set)
plt.axhline(chance_level, linestyle = '--', color = 'k')
plt.xlabel('Trials in session')
plt.ylabel('Reward rate')
plt.tight_layout()

In [255]:
# distance moved 
dist_mat = np.abs(np.diff(a, axis = 1))
dist_mat[dist_mat==0] = np.nan
sns.histplot(dist_mat.flatten(), discrete=True, stat= 'probability')

<Axes: ylabel='Probability'>

In [257]:
# transition matrix
parula = get_parula_cmap()
import pandas as pd
sns.heatmap(pd.crosstab(pd.Series(a.flatten()).shift(-1), a.flatten(), normalize= 'index'),
            cmap = parula,
            square=True, 
            # annot=True,
            vmax = 0.25,
            xticklabels=np.arange(1,5), 
            yticklabels=np.arange(1,5))
plt.xlabel('Choice at trial t+1')
plt.ylabel('Choice at trial t')
plt.tight_layout()

In [13]:
# plot behavior scatter in random subset of sessions
plt.plot(a[9:12].flatten(), '.', color = 'xkcd:cornflower')
plt.fill_between(np.arange(300),
                 np.repeat(np.argmax(rp_set[9:12], axis = 1)-0.5, 100), 
                 np.repeat(np.argmax(rp_set[9:12], axis = 1)+0.5, 100), 
                 alpha = 0.2, color = 'xkcd:cornflower')
plt.yticks([0, 1, 2, 3], [1,2,3,4])
plt.xlabel('Trials')
plt.ylabel('Choices')
plt.tight_layout()

# GPflow

In [ ]:
with open('L:/4portProb_processed/cleandf.pkl', 'rb') as f:
    df = pickle.load(f)

In [2]:
import pickle
import numpy as np
with open('L:/4portProb_processed/cleandf.pkl', 'rb') as f:
    df = pickle.load(f)

df[(df.animal == 'Raltz') & (df.task == 'unstr')]

ModuleNotFoundError: No module named 'numpy._core.numeric'

In [4]:
# GP flow implementation 
from typing import Sequence

import matplotlib.pyplot as plt
import numpy as np

import gpflow

c:\Users\dlab\miniforge3\envs\gpflow\lib\site-packages\gpflow\versions.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [5]:
X = np.load('L:/4portProb_processed/raltz_choices.npy').reshape(-1, 1)
Y = np.load('L:/4portProb_processed/raltz_reward.npy').reshape(-1, 1)

In [ ]:
X[:, :], Y[:4, :]

(array([[2.],
        [3.],
        [3.],
        [1.]]),
 array([[1.],
        [0.],
        [0.],
        [1.]]))

In [6]:
model = gpflow.models.VGP(
    (X, Y),
    kernel=gpflow.kernels.SquaredExponential(),
    likelihood=gpflow.likelihoods.Bernoulli(),
)
opt = gpflow.optimizers.Scipy()
opt.minimize(model.training_loss, model.trainable_variables)
gpflow.utilities.print_summary(model, "notebook")

ResourceExhaustedError: Graph execution error:

Detected at node 'MatrixBandPart/fill_triangular_CONSTRUCTED_AT_top_level/forward/fill_triangular/concat' defined at (most recent call last):
    File "c:\Users\dlab\miniforge3\envs\gpflow\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\dlab\miniforge3\envs\gpflow\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\dlab\miniforge3\envs\gpflow\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\dlab\miniforge3\envs\gpflow\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\dlab\miniforge3\envs\gpflow\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\dlab\miniforge3\envs\gpflow\lib\site-packages\tornado\platform\asyncio.py", line 211, in start
      self.as

In [54]:
model.predict_f_samples(Xplot, 10).numpy().squeeze().T

KeyboardInterrupt: 

In [44]:
Xplot = np.linspace(0, 1, 200)[:, None]
Fsamples = model.predict_f_samples(Xplot, 10).numpy().squeeze().T

plt.plot(Xplot, Fsamples, "C0", lw=0.5)

ShapeMismatchError: 
Tensor shape mismatch.
  Function: Kernel.__call__
    Declared: c:\Users\dlab\miniforge3\envs\gpflow\lib\site-packages\gpflow\kernels\base.py:188
    Argument: X
      Expected: [batch..., N, D]
      Actual:   [1, 21600]
    Argument: X2
      Expected: [batch2..., N2, D]
      Actual:   [200, 1]
